In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
dfs = []
for file in os.listdir('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_1/'):
    if (file!='dados.log' and file!='notas_finais.log' and (file=='54db0c6a27541070e9415a3d.csv' or file=='54db0c6a27541070e9415a3e.csv' or file=='54dbb8cc27541070e9415a41.csv')):
        dfs.append(pd.read_csv('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_1/'+file))
for file in os.listdir('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_2/'):
    if (file!='dados.log' and file!='notas_finais.log' and (file=='54db0c6a27541070e9415a3d.csv' or file=='54db0c6a27541070e9415a3e.csv' or file=='54dbb8cc27541070e9415a41.csv')):
        dfs.append(pd.read_csv('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_2/'+file))
for file in os.listdir('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_3/'):
     if (file!='dados.log' and file!='notas_finais.log' and (file=='54db0c6a27541070e9415a3d.csv' or file=='54db0c6a27541070e9415a3e.csv' or file=='54dbb8cc27541070e9415a41.csv')):
        dfs.append(pd.read_csv('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_3/'+file))
for file in os.listdir('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_4/'):
     if (file!='dados.log' and file!='notas_finais.log' and (file=='54db0c6a27541070e9415a3d.csv' or file=='54db0c6a27541070e9415a3e.csv' or file=='54dbb8cc27541070e9415a41.csv')):
        dfs.append(pd.read_csv('/home/samuel/Documentos/antigo/estey_coady/por_média/semestre_4/'+file))

In [3]:
df=pd.concat(dfs)
df=df.groupby('Id_aluno', as_index=False).mean()
#df['final_exam']=dfs[0]['final_exam']
#df=pd.read_csv('/home/samuel/estey_coady/por_média/semestre_3/54db0c6a27541070e9415a3d.csv')

In [4]:
def normalizacao(df):
    from sklearn import preprocessing
    np_scaled = preprocessing.scale(df)
    df_normalized = pd.DataFrame(np_scaled, columns = df.columns)
    return df_normalized

In [5]:
def discretizacao_das_medias(df):
    df.loc[df.final_exam<50.0, 'final_exam'] = 0
    df.loc[df.final_exam>=50.0, 'final_exam'] = 1
    df['final_exam'] = df.final_exam.astype(int)
    return df

In [6]:
def subamostragem_balanceamento(df):
    g = df.groupby('final_exam')
    df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
    return df

In [7]:
colunas = ['Id_aluno']
df= df.drop(labels = colunas, axis=1)
df.loc[:, df.columns != 'final_exam'] = normalizacao(df.loc[:, df.columns != 'final_exam'])
df = discretizacao_das_medias(df)
df.head()

,compiles,correct_attempts,error,hints,runs,total_attempts,time,final_exam
0,-0.123451,-0.171905,-0.381206,-0.187535,-0.130001,-0.163563,-0.400679,0
1,-0.123451,-0.171905,-0.381206,-0.187535,-0.130001,-0.163563,-0.400679,0
2,-0.123451,-0.171905,-0.381206,-0.187535,-0.130001,-0.163563,-0.400679,0
3,-0.123451,-0.171905,-0.381206,-0.187535,-0.130001,-0.163563,-0.400679,0
4,-0.040883,-0.024998,-0.312122,9.193197,-0.026199,-0.004411,3.533405,0


In [8]:
X, y = df.drop('final_exam', axis=1).values, df['final_exam'].values

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
from numpy import loadtxt
# from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


# params = {  
#       'nthread':[4,5,6], #when use hyperthread, xgboost may become slower
#       'objective':['binary:logistic'],
#       'learning_rate': [0.05,0.06,0.07], #so called `eta` value
#       'max_depth': [5,6,7,8,9],
#       'min_child_weight': [9,10,11,12,13],
#       'silent': [1,2,3],
#       'subsample': [0.8,0.9],
#       'colsample_bytree': [0.7,0.8,0.9],
#       'n_estimators': [4,5,6], #number of trees, change it to 1000 for better results
#       'missing':[-999],
#       'seed': [1337],
#       'early_stopping':[True],
# }

# knn = XGBClassifier()
# grid = RandomizedSearchCV(knn, params, cv=10, n_jobs=-1, verbose=1, n_iter=10) #n_jobs = -1 faz a computação em paralelo
# grid.fit(X,y)

# print(grid.best_params_)
# print(grid.best_estimator_)
# print(grid.best_score_)

In [ ]:
params1 = {  
    "bootstrap":[True],
    "criterion":['entropy','gini'], 
    "max_features": ['auto', 'sqrt', 'log2'],
    "min_samples_leaf":[5,6,7,8,9], 
    "min_samples_split":[16,17,18,19,20,21],
    "max_depth" : [4,5,6,7,8],
    "class_weight":['balanced','balanced_subsample'],
}

knn1 = ExtraTreesClassifier()
grid1 = RandomizedSearchCV(knn1, params1, cv=10, n_jobs=-1, verbose=1, n_iter=1000) #n_jobs = -1 faz a computação em paralelo
grid1.fit(X,y)

print(grid1.best_params_)
print(grid1.best_estimator_)
print(grid1.best_score_)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1747 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 5247 tasks      | elapsed:   14.0s


In [ ]:
params2 = {  
    "bootstrap":[True],
    "criterion":['entropy','gini'], 
    "max_features": ['auto', 'sqrt', 'log2'],
    "min_samples_leaf":[5,6,7,8,9], 
    "min_samples_split":[16,17,18,19,20,21],
    "max_depth" : [4,5,6,7,8],
    "class_weight":['balanced','balanced_subsample'],
}
knn2 = RandomForestClassifier()
grid2 = RandomizedSearchCV(knn2, params2, cv=10, n_jobs=-1, verbose=1, n_iter=1000) #n_jobs = -1 faz a computação em paralelo
grid2.fit(X,y)

print(grid2.best_params_)
print(grid2.best_estimator_)
print(grid2.best_score_)

In [ ]:
from __future__ import absolute_import
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_digits as load_data
import scikitplot as skplt


X, y = df.drop('final_exam', axis=1).values, df['final_exam'].values
nb = GaussianNB()
nb.fit(X, y)
probas = nb.predict_proba(X)
skplt.metrics.plot_roc(y_true=y, y_probas=probas)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.feature_selection import RFE


#pipeline = make_pipeline(
#            RFE(estimator=ExtraTreesClassifier(criterion="gini", n_estimators=100), step=0.5),
#            RandomForestClassifier(max_features=0.9500000000000001, min_samples_leaf=20, n_estimators=500)
#        )

#pipeline = RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.45, min_samples_leaf=7, min_samples_split=19,class_weight='balanced_subsample')
pipeline =grid2.best_estimator_

In [ ]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(pipeline, X, y, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#clf=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X,y,cv=10)
print(confusion_matrix(y,y_pred))


In [ ]:
print(classification_report(y,y_pred))

In [ ]:
import imblearn

In [ ]:
from sklearn.datasets import make_classification

#df = pd.DataFrame(X)
#df['target'] = y
cont_target = df.final_exam.value_counts()
cont_target.plot(kind='bar', color=['blue', 'red'], title='Distribuição das notas');

In [ ]:
def plot_2d_space(X, y, label='Classes'):   
    colors = ['#1F77B4', '#FF7F0E']
    markers = ['o', 's']
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(
            X[y==l, 0],
            X[y==l, 1],
            c=c, label=l, marker=m
        )
    plt.title(label)
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
a = pca.fit_transform(X)

plot_2d_space(a, y, 'Base de dados desbalanceada (2 PCA components)')

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(return_indices=True)
X_rus, y_rus, id_rus = rus.fit_sample(X, y)

print('Índices removidos:', id_rus)

plot_2d_space(X_rus, y_rus, 'Random Undersampling')

In [ ]:
nb = GaussianNB()
nb.fit(X_rus, y_rus)
probas = nb.predict_proba(X_rus)
skplt.metrics.plot_roc(y_true=y_rus, y_probas=probas)
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(pipeline, X_rus, y_rus, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#clf1=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_rus,y_rus,cv=10)
print(confusion_matrix(y_rus,y_pred))

In [ ]:
print(classification_report(y_rus,y_pred))

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X, y)

print(X_ros.shape[0] - X.shape[0], 'new random picked points')
print(X_ros.shape[0] - X.shape[0], 'novos pontos aleatórios selecionados')

plot_2d_space(X_ros, y_ros, 'Random Oversampling')

In [ ]:
nb = GaussianNB()
nb.fit(X_ros, y_ros)
probas = nb.predict_proba(X_ros)
skplt.metrics.plot_roc(y_true=y_ros, y_probas=probas)
plt.show()

In [ ]:
cv_score = cross_val_score(pipeline, X_ros, y_ros, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
clf2=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_ros,y_ros,cv=10)
print(confusion_matrix(y_ros,y_pred))

In [ ]:
print(classification_report(y_ros,y_pred))

In [ ]:
from imblearn.under_sampling import TomekLinks

tl = TomekLinks(return_indices=True, sampling_strategy='majority')
X_tl, y_tl, id_tl = tl.fit_sample(X, y)

print('Removed indexes:', id_tl)

plot_2d_space(X_tl, y_tl, 'Tomek links under-sampling')

In [ ]:
nb = GaussianNB()
nb.fit(X_tl, y_tl)
probas = nb.predict_proba(X_tl)
skplt.metrics.plot_roc(y_true=y_tl, y_probas=probas)
plt.show()

In [ ]:
cv_score = cross_val_score(pipeline, X_tl, y_tl, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
clf=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_tl,y_tl,cv=10)
print(confusion_matrix(y_tl,y_pred))

In [ ]:
print(classification_report(y_tl,y_pred))

In [ ]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids()
X_cc, y_cc = cc.fit_sample(X, y)

plot_2d_space(X_cc, y_cc, 'Cluster Centroids Undersampling')

In [ ]:
cont=0
a=[]
while(cont<len(X_cc)):
    flag=1
    for cada in X_cc[cont]:
        if cada!=0:
            flag=0
    if flag==0:
        a+=[[X_cc[cont][0]]+[X_cc[cont][1]]+[X_cc[cont][2]]+[X_cc[cont][3]]+[X_cc[cont][4]]+[X_cc[cont][5]]+[X_cc[cont][6]]+[y_cc[cont]]]
    cont+=1
#a=np.array(a)
#a[0]

pd.DataFrame(a).to_csv("help.csv")
df=pd.read_csv('help.csv')
df=df.drop(df.columns[[0]], axis=1)
df['final_exam']=df['7']
df=df.drop(df.columns[[-2]], axis=1)
X_cc, y_cc = df.drop('final_exam', axis=1).values, df['final_exam'].values

In [ ]:
nb = GaussianNB()
nb.fit(X_cc, y_cc)
probas = nb.predict_proba(X_cc)
skplt.metrics.plot_roc(y_true=y_cc, y_probas=probas)
plt.show()

In [ ]:
cv_score = cross_val_score(pipeline, X_cc, y_cc, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#clf=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_cc,y_cc,cv=10)
print(confusion_matrix(y_cc,y_pred))

In [ ]:
print(classification_report(y_cc,y_pred))

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', k_neighbors=5)
X_sm, y_sm = smote.fit_sample(X, y)

plot_2d_space(X_sm, y_sm, 'SMOTE over-sampling')

In [ ]:
nb = GaussianNB()
nb.fit(X_sm, y_sm)
probas = nb.predict_proba(X_sm)
skplt.metrics.plot_roc(y_true=y_sm, y_probas=probas)
plt.show()

In [ ]:
cv_score = cross_val_score(pipeline, X_sm, y_sm, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
clf=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_sm,y_sm,cv=10)
print(confusion_matrix(y_sm,y_pred))

In [ ]:
print(classification_report(y_sm,y_pred))

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', k_neighbors=3)
X_sm, y_sm = smote.fit_sample(X, y)

plot_2d_space(X_sm, y_sm, 'SMOTE over-sampling')

In [ ]:
nb = GaussianNB()
nb.fit(X_sm, y_sm)
probas = nb.predict_proba(X_sm)
skplt.metrics.plot_roc(y_true=y_sm, y_probas=probas)
plt.show()

In [ ]:
cv_score = cross_val_score(pipeline, X_sm, y_sm, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
clf=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_sm,y_sm,cv=10)
print(confusion_matrix(y_sm,y_pred))

In [ ]:
print(classification_report(y_sm,y_pred))

In [ ]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(sampling_strategy='auto')
X_smt, y_smt = smt.fit_sample(X, y)

plot_2d_space(X_smt, y_smt, 'SMOTE + Tomek links')

In [ ]:
nb = GaussianNB()
nb.fit(X_smt, y_smt)
probas = nb.predict_proba(X_smt)
skplt.metrics.plot_roc(y_true=y_smt, y_probas=probas)
plt.show()

In [ ]:
cv_score = cross_val_score(pipeline, X_smt, y_smt, scoring='accuracy', cv=10)

In [ ]:
print('Acc.: %.2f [+/-%.2f]' % (cv_score.mean(), cv_score.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
clf=RandomForestClassifier(n_estimators=100)
y_pred=cross_val_predict(pipeline,X_smt,y_smt,cv=10)
print(confusion_matrix(y_smt,y_pred))

In [ ]:
print(classification_report(y_smt,y_pred))
df.head()

In [ ]:
#a=[]#[['compiles','correct_attempts','error','hints','runs','total_attempts','time','final_exam']]
cont=0
a=[]
X_ros=X_cc
y_ros=y_cc
# X_cc=X
# y_cc=y
while(cont<len(X_ros)):
    flag=1
    for cada in X_ros[cont]:
        if cada!=0:
            flag=0
    if flag==0:
        a+=[[X_ros[cont][0]]+[X_ros[cont][1]]+[X_ros[cont][2]]+[X_ros[cont][3]]+[X_ros[cont][4]]+[X_ros[cont][5]]+[X_ros[cont][6]]+[y_ros[cont]]]
    cont+=1
# while(cont<len(X_cc)):
   
#         a+=[[X_cc[cont][0]]+[X_cc[cont][1]]+[X_cc[cont][2]]+[X_cc[cont][3]]+[X_cc[cont][4]]+[X_cc[cont][5]]+[X_cc[cont][6]]+[y_cc[cont]]]
#         cont+=1
#a=np.array(a)
#a[0]

pd.DataFrame(a).to_csv("help.csv")
df=pd.read_csv('help.csv')
df=df.drop(df.columns[[0]], axis=1)
df['final_exam']=df['7']
df=df.drop(df.columns[[-2]], axis=1)
# df.loc[:, df.columns != 'final_exam'] = normalizacao(df.loc[:, df.columns != 'final_exam'])
housing = df.drop("final_exam", axis=1)
housing_labels = df["final_exam"].copy()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

lin_reg = LinearRegression()
tree_reg = DecisionTreeRegressor(random_state=42)
forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)

def cross_val(estimator,X, y):
    scores = cross_val_score(estimator, X, y,
                         scoring="neg_mean_squared_error", cv=10)
    return np.sqrt(-scores)

tree_rmse_scores = cross_val(tree_reg, housing, housing_labels)

lin_reg_scores = cross_val(lin_reg, housing, housing_labels)

forest_reg_rmse_scores = cross_val(forest_reg, housing, housing_labels)
def display_scores(scores, alg_name):
    print(alg_name)
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    print('---------------------------------')

display_scores(tree_rmse_scores, 'DecisionTreeRegressor')
display_scores(lin_reg_scores, 'LinearRegression')
display_scores(forest_reg_rmse_scores, 'RandomForestRegressor')

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing, housing_labels)

print(grid_search.best_params_)

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
x_pos=np.array([1, 7, 12, 14,16,20,24])
print(x_pos)
bar_1=feature_importances[0]
bar_2=feature_importances[1]
bar_3=feature_importances[2]
bar_4=feature_importances[3]
bar_5=feature_importances[4]
bar_6=feature_importances[5]
bar_7=feature_importances[6]
plt.bar([1], bar_1)
plt.bar([7], bar_2)
plt.bar([12], bar_3)
plt.bar([14], bar_4)
plt.bar([16], bar_5)
plt.bar([20], bar_6)
plt.bar([24], bar_7)

plt.xticks(x_pos+0.25,('compiles','correct_attempts','error','hints','runs','total_attempts','time'))
plt.show()

In [ ]:
labels=np.array(['compiles', 'correct_attempts', 'error_free_compiles', 'hints', 'runs', 'total_attempts','time'])
cont=0
passou=0
npassou=0
stats1=[0,0,0,0,0,0,0]
stats2=[0,0,0,0,0,0,0]
for elemento in housing_labels:
    if elemento==0:
        stats1[0]+=housing.loc[cont,['0']].values[0]
        stats1[1]+=housing.loc[cont,['1']].values[0]
        stats1[2]+=housing.loc[cont,['2']].values[0]
        stats1[3]+=housing.loc[cont,['3']].values[0] 
        stats1[4]+=housing.loc[cont,['4']].values[0]
        stats1[5]+=housing.loc[cont,['5']].values[0]
        stats1[6]+=housing.loc[cont,['6']].values[0]
        npassou+=1
    else:
        stats2[0]+=housing.loc[cont,['0']].values[0]
        stats2[1]+=housing.loc[cont,['1']].values[0]
        stats2[2]+=housing.loc[cont,['2']].values[0]
        stats2[3]+=housing.loc[cont,['3']].values[0] 
        stats2[4]+=housing.loc[cont,['4']].values[0]
        stats2[5]+=housing.loc[cont,['5']].values[0]
        stats2[6]+=housing.loc[cont,['6']].values[0]
        passou+=1
    cont+=1
            
cont=0
while(cont<len(stats1)):
    stats1[cont]=stats1[cont]/npassou
    cont+=1
cont=0
while(cont<len(stats2)):
    stats2[cont]=stats2[cont]/passou
    cont+=1
stats1=np.asarray(stats1)
stats2=np.asarray(stats2)
print(stats1)
print(stats2)


In [ ]:
angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
# close the plot
stats1=np.concatenate((stats1,[stats1[0]]))
angles=np.concatenate((angles,[angles[0]]))
stats2=np.concatenate((stats2,[stats2[0]]))


In [ ]:
ax = plt.subplot(111, polar=True)
ax.plot(angles, stats1, 'o-', linewidth=2,color='red')
ax.fill(angles, stats1, alpha=0.25,color='red')
ax.set_thetagrids(angles * 180/np.pi, labels,color='black')
ax.grid(True)


bx = plt.subplot(111, polar=True)
bx.plot(angles, stats2, 'o-', linewidth=2,color='green')
bx.fill(angles, stats2, alpha=0.25,color='green')
bx.set_thetagrids(angles * 180/np.pi, labels)
bx.grid(True)
# lista_foto=['eps', 'jpeg', 'jpg', 'pdf', 'png', 'ps', 'raw', 'rgba', 'svg', 'svgz', 'tif', 'tiff']
# for cada in lista_foto:
#     plt.savefig('/home/samuel/Imagens/radar.'+cada, format=cada)